In [1]:
import sys
from multiprocessing import Pool,cpu_count

import numpy as np
np.set_printoptions(threshold=np.nan)

import random

import pandas as pd
pd.set_option('display.max_rows', None)


from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor


import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D

In [2]:
class probabilityFunction:

    def __init__(self,opt_space,iteration,sample_id,objective_,optimizer,search_based,hybrid,RF_depth,RF_estimator,no_selections):
        
        self.space = opt_space


        self.based = search_based

        self.iteration = iteration
        self.evaluated = self.space[self.space['objective'].notnull()].index.values
        self.sample_id = sample_id
        self.objective = objective_

        self.depth = RF_depth
        self.estimator = RF_estimator
        self.no_selections = no_selections
        
        self.pd_header = list(self.space)
        self.no_variables = len(self.pd_header) - len(sys_information)
        

        self.mesh_points, self.tab_columns = self.space.shape


    # ================================
    # ================================
    def hybridWAM(self):
        global prob_model_r2
        global prob_model

        learning_limit = 0.99

        probability = np.zeros(self.mesh_points)


        # Check learning status and use random selection
        if((prob_model_r2 <= learning_limit) or (len(self.evaluated) < 0.01*self.mesh_points)):
            
            inputs_ = self.space[self.pd_header[:self.no_variables]].loc[self.evaluated].as_matrix().reshape((-1,self.no_variables))
            outputs_ = self.space['objective'].loc[self.evaluated].as_matrix().flatten()

            prob_model = RandomForestRegressor(max_depth=len(self.evaluated), n_estimators=self.estimator)  #, n_jobs=-1)
            prob_model.fit(inputs_, outputs_)
            probability_ = prob_model.predict(inputs_)


            prob_model_r2 = r2_score(outputs_,probability_)

            learning_curve.append([self.iteration,len(self.evaluated),prob_model_r2])


        # use RF-WAM function
        if(prob_model_r2 > learning_limit):
            
            #probability = self.randomforest()
            inputs_ = self.space[self.pd_header[:self.no_variables]].as_matrix().reshape((-1,self.no_variables))
            probability_ = prob_model.predict(inputs_)

            prob_ = probability_ * self.space["weighting"].as_matrix()
            
            prob_matrix = np.stack((self.space.index.values,prob_), axis=-1)
            prob_matrix = np.delete(prob_matrix,np.where(prob_matrix[:,1] == 0)[0],axis=0)
            

            selected_index = prob_matrix[prob_matrix[:, 1].argsort()][::-1][:self.no_selections,0].astype(int)


            probability[selected_index] = 1.
            



        return probability

    # ================================
    # ================================



    def randomforest(self):

    # train randomforest model for probability function

        # the for loop cost 420.534-356.826 more seconds
        '''
        inputs_ = self.space[self.pd_header[0]].loc[self.evaluated].as_matrix().reshape((-1,1))

        for header_ in self.pd_header[1:-3]:
            inputs_ = np.hstack((inputs_,self.space[header_].loc[self.evaluated].as_matrix().reshape((-1,1))))
        '''

        inputs_ = self.space[self.pd_header[:self.no_variables]].loc[self.evaluated].as_matrix().reshape((-1,self.no_variables))

        outputs_ = self.space['objective'].loc[self.evaluated].as_matrix().flatten()

        #prob_model = RandomForestRegressor(max_depth=self.depth, n_estimators=self.estimator)
        prob_model = RandomForestRegressor(max_depth=len(self.evaluated), n_estimators=self.estimator)  #, n_jobs=-1)
        prob_model.fit(inputs_, outputs_)


        '''
        # to estimate the uncertainty (confidence interval)
        calculated_table = []
        for pred in prob_model.estimator:
            calculated_table.append(pred.predict(inputs_))
        '''


        # use randomforest model to estimate the probability function
        '''
        inputs_ = self.space[self.pd_header[0]].as_matrix().reshape((-1,1))

        for header_ in self.pd_header[1:-3]:
            inputs_ = np.hstack((inputs_,self.space[header_].as_matrix().reshape((-1,1))))
        '''

        inputs_ = self.space[self.pd_header[:self.no_variables]].as_matrix().reshape((-1,self.no_variables))


        # =====================================================================
        # the selsction mechanisms are very different
        # 'monte-carlo' uses probability for selection
        # 'whack-a-mole' sorts probability matrix and uses the highest points
        # =====================================================================
        if( self.based == 'monte-carlo'):
            probability = prob_model.predict(inputs_)

        # while the objective values are negative
        # the probabilities are negative
        # the curve is therefore shifted above the zero
        # this is different from probability matrix
            probability += np.amin(probability)

        # if objective is negative, the probability will be negative
        # this maximization function enforces 0 on probability
            probability[probability<0] = 0.



        elif(self.based =='whack-a-mole'):

            probability = np.zeros(self.mesh_points)
            
            probability_ = prob_model.predict(inputs_)
            prob_matrix = probability_ * self.space["weighting"].as_matrix()


            # while the objective values are negative
            # the probability is negative
            # in probability matrix, the calculated points possess the highest objective value =0
            # which are removed from probability matrix

            prob_ = probability_ * self.space["weighting"].as_matrix()
            
            prob_matrix = np.stack((self.space.index.values,prob_), axis=-1)
            prob_matrix = np.delete(prob_matrix,np.where(prob_matrix[:,1] == 0)[0],axis=0)
            

            selected_index = prob_matrix[prob_matrix[:, 1].argsort()][::-1][:self.no_selections,0].astype(int)

            probability[selected_index] = 1.

            

        else:
            sys.exit("illeagle setting parameter: RF_search_based")


        return probability



    def geneticalgorithm(self):

        global GA_search_top
        global GA_search_path


        if(len(self.evaluated)>=self.mesh_points):
            return np.full(self.mesh_points,0.)

        elif(len(self.evaluated)>=self.mesh_points-self.no_selections):
            return np.full(self.mesh_points,1.)




        # calculate required bits of a string for each variable
        no_bits = np.zeros(self.no_variables+1,dtype=int)
        binary_max = []

        for ivar in range(self.no_variables):

            bin_ = bin(no_gridpoints[ivar])[2:]
            no_bits[ivar+1] = len(bin_)

            binary_max.append(2**no_bits[ivar+1])




        if(self.based == 'family'):

            no_objective_ = GA_no_parents
            if(len(self.objective) < GA_no_parents):
                no_objective_ = len(self.objective)


            # obtain the maximum objective from current iteration
            objective = sorted(self.objective, reverse=True)[:no_objective_]


            # obtain the id from table
            # GA-family is path dependent, 
            # so the selected samples will be passed to the next generation
            #
            id_ = []
            for iobjective in range(no_objective_):
                try:
                    id_.extend([self.sample_id[self.objective.index(objective[iobjective])]])

                except:

                    print(iobjective,objective,self.objective,self.sample_id)
                    print(len(self.evaluated),self.mesh_points)


                    sys.exit("error from GA family method")



            # obtain the maximum objective from previous iteration
            if GA_search_path:
                objective.extend(GA_search_top)
                id_.extend(GA_search_path[-1][1:])



            objective_top = sorted(objective, reverse=True)[:GA_no_parents]            
            GA_search_top = objective_top



            GA_search_path.append([self.iteration])
            for iobjective in range(GA_no_parents):
                GA_search_path[-1].extend([id_[objective.index(objective_top[iobjective])]])



        elif(self.based == 'global'):

            objective = np.sort(self.space['objective'].fillna(value=0.))[::-1]


            GA_search_path.append([self.iteration])

            for iobjective in range(GA_no_parents):

                index_ = self.space[self.space['objective'] == objective[iobjective] ].index.values
                GA_search_path[-1].extend(index_)

        else:
            sys.exit("illeagle setting parameters: GA_search_based")





        bitcount = np.cumsum(no_bits)
        # get positions of parents in axes in binary strings
        ga_parents = []
        for imax in range(GA_no_parents):

            selections = [GA_search_path[-1][imax+1]]



            variable_ = self.space[self.pd_header[0]].loc[selections].values[0]
            bin_variable_ = bin(np.where(var_space[0] == variable_)[0][0])[2:].zfill(no_bits[1])
            ga_parents_ = bin_variable_


            for ivar in range(1,self.no_variables):

                header_ = self.pd_header[ivar]

                variable_ = self.space[header_].loc[selections].values[0]

                # convert the best to binary string
                bin_variable_ = bin(np.where(var_space[ivar] == variable_)[0][0])[2:].zfill(no_bits[ivar+1])

                ga_parents_ += bin_variable_
                #ga_parents_ = np.hstack((ga_parents_,bin_variable_))


            ga_parents.append(ga_parents_)


        total_bits = len(ga_parents[0])


        similarity = 0.
        for ibits in range(total_bits):
            if ga_parents[0][ibits] == ga_parents[1][ibits]:
                similarity += 1.


    # ==== convergence/similarity test
        if (similarity/total_bits >= GA_convergence) or (similarity >= total_bits-1):
            probability = np.full(self.mesh_points,1.)

            GA_search_top = [0]*GA_no_parents


        else:

            no_children = self.no_selections - GA_no_parents


            opt_index = []
            opt_attempts = 0
            while (not opt_index):

                for ichild in range(no_children):

                    # selection for cross-over
                    randomselection = sorted(random.sample(range(0, total_bits), 2))


                    # cross-over
                    children_ = (ga_parents[0][0:randomselection[0]] 
                                 + ga_parents[1][randomselection[0]:randomselection[1]] 
                                 + ga_parents[0][randomselection[1]:])


                    # mutation
                    randomselection = random.sample(range(0, len(ga_parents[0])), int(mutation_rate*total_bits))



                    children_ = (children_[:randomselection[0]] 
                                 + str(abs(int(children_[randomselection[0]]) - 1)) 
                                 + children_[randomselection[0]+1:])


                    # convert back to search space
                    children = []
                    for ivar in range(self.no_variables):

                        binary_ = children_[bitcount[ivar]:bitcount[ivar+1]]

                        position_ = int(float(int(binary_, 2))/binary_max[ivar] * no_gridpoints[ivar])


                        value_ = var_space[ivar][position_]


                        children.append(self.space[self.space[self.pd_header[ivar]] == value_].index.values)


                    opt_index_ = children[0]
                    for ivar in range(1,self.no_variables):
                        opt_index_ = list(set(opt_index_).intersection(children[ivar]))

                    opt_index.append(opt_index_[0])



                opt_index = sorted(list(set(opt_index)))


                check_probability = sorted(list(set(self.evaluated).intersection(opt_index)))

                # ==== convergence test
                # if the selected inputs are calculated, it turns into random selection
                #
                if check_probability == opt_index:

                    #print("1",self.iteration)
                    opt_attempts += 1

                    if opt_attempts == 10:
                        probability = np.full(self.mesh_points,1.)

                        GA_search_top = [0]*GA_no_parents

                    else:
                        opt_index = []



                # the probability matrix (prob_matrix) must larger than GA_no_parents
                # eq: 0,x,1 to get at least 2 calculations as parent samples
                #

                elif( (self.mesh_points - len(self.evaluated) > GA_no_parents) 
                     & (len(opt_index) - len(check_probability) < (GA_no_parents+1)) ):

                    #print("2",self.iteration)
                    opt_attempts += 1

                    if opt_attempts == 10:
                        probability = np.full(self.mesh_points,1.)

                        GA_search_top = [0]*GA_no_parents

                    else:
                        opt_index = []


                elif (self.mesh_points - len(self.evaluated) > GA_no_parents) & (len(opt_index) == 1):

                    #print("3",self.iteration)
                    opt_attempts += 1

                    if opt_attempts == 10:
                        probability = np.full(self.mesh_points,1.)

                        GA_search_top = [0]*GA_no_parents

                    else:
                        opt_index = []              


                else:
                    probability = np.zeros(self.mesh_points)
                    probability[opt_index] = 1.


                    #prob_matrix = probability * self.space["weighting"].as_matrix()



        # while GA is converged, the probability function will be generated using RF-mc
        if((hybrid == 1) & (np.round(np.mean(probability),6) == 1.)):
            self.based = 'monte-carlo'

            probability = self.randomforest()


        # hybrid_3 is the combination of GA-family + 'RF-WAM'
        if(hybrid == 3):
            probability_ = self.hybridWAM()

            if(np.count_nonzero(probability_) > 0):
                probability = probability_




        return probability




    def random(self):
        probability = np.full(self.mesh_points,1.)


        if(hybrid == 2):
            probability_ = self.hybridWAM()

            if(np.count_nonzero(probability_) > 0):
                probability = probability_



        return probability

In [3]:
def optimization(evaluate,probFunctionName,noIteration,opt_space,RF_depth,RF_estimator,no_selections,saveSpace):

    pd_header = list(opt_space)
    no_variables = len(pd_header) - len(sys_information)
    mesh_points, tab_columns = opt_space.shape  
    
    
    prob_matrix = np.zeros(mesh_points)


    #ML_matrix = [np.zeros(mesh_points)]
    for iteration in range(noIteration):

        evaluated = opt_space[opt_space['objective'].notnull()].index.values
        
        #print(iteration,len(evaluated),prob_model_r2)
        #print(iteration,len(evaluated))
        probsum = opt_space['objective'].sum()
        #print("----",len(evaluated), mesh_points)


    # ====== select sample by the probability function
        # probsum == 0 is the condition for constrain search
        if((len(evaluated) >= mesh_points) or (probsum == 0) ):
            #print("the space is fully evaluated: %d points" %len(evaluated))
            return opt_space


        else:

            # first iteration of search
            # the probability column is nan
            if(len(evaluated)==0):
                sampling = np.linspace(0.,1.-0.5/mesh_points,no_selections,endpoint=True)

                sample_id = []
                for isam in range(len(sampling)):
                    sample_id.append(np.where(opt_space['probability']>sampling[isam])[0][0])


                sample_id = np.unique(np.array(sample_id))


            else:

                prob_ = np.unique(opt_space['probability'])


                # for last iteration:
                # all the remaining points are selected
                if(len(evaluated) >= mesh_points-no_selections):
           
                    sample_id = opt_space[opt_space['objective'].isnull()].index.values


                # the optimizers assign the probability to each grid point 
                # and the algorithm choose samples according to the probability
                # for assigned sampling, random selection will cause lower calculation rate
                elif(len(prob_) <= no_selections):

                    sample_id = []
                    for isam in range(1,len(prob_)):
                        sample_id.append(np.where(opt_space['probability']==prob_[isam])[0][0])


                else:
            
                    sampling = np.random.rand(1,no_selections)[0]

                    sample_id = []
                    for isam in range(len(sampling)):
                        sample_id.append(np.where(opt_space['probability']>sampling[isam])[0][0])


                    sample_id = np.unique(np.array(sample_id))


        test_sample = np.around(opt_space[pd_header[:no_variables]].loc[sample_id].values, decimals=10)


        
        # ====== sample evaluation
        objective_ = []
        for isam in range(len(test_sample)):
            objective_.append(evaluate(test_sample[isam]))


            #print("OPT test sample: ",test_sample[isam])
            #print(objective_[-1])


            #opt_space['objective'].set_value(sample_id[isam], objective_[-1])
            #opt_space['weighting'].set_value(sample_id[isam], 0.)
            opt_space['objective'].iat[sample_id[isam]] = objective_[-1]
            opt_space['weighting'].iat[sample_id[isam]] = 0.
            opt_space['iteration'].iat[sample_id[isam]] = iteration

        # ====== sample evaluation
        
        ''' # -- Parallel processing
        processes = cpu_count()
        if(processes == 0):
          processes = 1

        tasks = len(test_sample)//processes

        with Pool(processes) as p:
            objective_ = p.map(evaluate, test_sample, chunksize=tasks)
        
        
        opt_space.loc[sample_id,['objective']] = objective_
        opt_space.loc[sample_id,['weighting']] = 0.
	'''


        prob = probabilityFunction(opt_space,iteration,sample_id,objective_,optimizer,search_based,hybrid,RF_depth,RF_estimator,no_selections)
        functionName = getattr(prob,probFunctionName)
        ML_matrix = functionName()



        prob_matrix = ML_matrix * opt_space["weighting"].as_matrix()


        cum_prob_matrix = np.cumsum(prob_matrix)
        max_prob = cum_prob_matrix[-1]


        if max_prob == 0:
            opt_space['probability'] = prob_matrix
        else:
            opt_space['probability'] = cum_prob_matrix/max_prob


        if(saveSpace == 'save'):
            opt_space.to_csv('opt_space.csv', index=False)



    return opt_space

In [4]:
def run(readfile,searchMesh,evaluate,optimization_method,no_selections,noIteration,saveSpace):
#==========================================

    no_variables = len(searchMesh)


    global learning_curve
    learning_curve = []

# ===============================================================================
# Create Mesh
# ===============================================================================
    # ======= inputs to the algorithm
    #
    # initiate pandas dataframe
    #
    pd_header = []
    for ivar in range(no_variables):
        pd_header.extend(['input' + str(ivar)])


    global sys_information
    sys_information = ['objective','probability','weighting','iteration']
    #sys_information = ['objective','probability','weighting']
    pd_header.extend(sys_information)
# ===============================================================================

    grid_type = []
    grid_bias_ = []
    search_space = []
    # -------------- create mesh
    for ivar in range(no_variables):
        grid_type.append(searchMesh[ivar][0])


        # only for 'polynomial' method
        if(grid_type[-1] == 'polynomial'):
            grid_bias_.append(searchMesh[ivar][1]) # [0.~-1., order]

            # define search space
            search_space.append(searchMesh[ivar][2])

        else:
            # define search space
            search_space.append(searchMesh[ivar][1])



    search_space = np.array(search_space)

    if ('log' in grid_type) & (np.any(search_space[:,0] == 0.) ):
        sys.exit("illegal space variable for for 'log' method")

# ===============================================================================

    optimization_method = optimization_method.lower()


    try:
        ind_ = optimization_method.index('-')
    except:
        ind_ = -1


# ============================================================
    global hybrid,optimizer,search_based 
    
    hybrid = 0
    optimizer = ''
    search_based = ''

    if(ind_ == -1):
        if(optimization_method == 'hybrid_1'):
            hybrid = 1
            optimizer = 'geneticalgorithm'
            search_based = 'global'

        elif(optimization_method == 'hybrid_2'):
            hybrid = 2
            optimizer = 'random'
            search_based = 'whack-a-mole'

        elif(optimization_method == 'hybrid_3'):
            hybrid = 3
            optimizer = 'geneticalgorithm'
            search_based = 'family'

        elif(optimization_method == 'random'):
            optimizer = optimization_method

        else:
            sys.exit("illegal variable optimization_method")

    else:
        #print(optimization_method[:ind_])
        if(optimization_method[:ind_] == 'rf'):
            optimizer = 'randomforest'

            if(optimization_method[ind_+1:] == 'wam'):
                search_based = 'whack-a-mole'
            elif(optimization_method[ind_+1:] == 'mc'):
                search_based = 'monte-carlo'

        elif(optimization_method[:ind_] == 'ga'):
            optimizer = 'geneticalgorithm'
            search_based = optimization_method[ind_+1:]


    # -------------- parameters for genetic algorithm
    global GA_no_parents,mutation_rate,GA_convergence
    GA_no_parents = 2
    mutation_rate = 0.3
    GA_convergence = 0.9

    if (optimizer == 'geneticalgorithm') & (no_selections == 2):
        sys.exit("illegal number selection for each iteration for for genetic algorithm")
    # --------------

    # -------------- parameters for randomforest
    RF_depth = no_selections*noIteration
    RF_estimator = 100
    # --------------

    # ======= 

    global no_gridpoints,var_space
    no_gridpoints = []
    var_space = []
    for ivar in range(no_variables):

        no_gridpoints.append(int(np.trunc((search_space[ivar,1]-search_space[ivar,0])/search_space[ivar,2])) + 1)

        if grid_type[ivar].lower() == 'polynomial':


            grid_bias = grid_bias_[ivar][0]*no_gridpoints[-1]


            mesh_step = np.linspace(0, no_gridpoints[-1], no_gridpoints[-1], endpoint=True)
            mesh_function = np.cumsum((mesh_step-grid_bias)**grid_bias_[ivar][1])

            dist_ = (search_space[0,0] + (search_space[0,1]-search_space[0,0])
                     * (mesh_function-mesh_function[0])/(mesh_function[-1]-mesh_function[0]))

            var_space.append(dist_)

        elif grid_type[ivar].lower() == 'linear':
            var_space.append(np.linspace(search_space[ivar,0], search_space[ivar,1], num=no_gridpoints[-1]))

        elif grid_type[ivar].lower() == 'log':
            var_space.append(np.geomspace(search_space[ivar,0], search_space[0,1], num=no_gridpoints[-1]))

        else:
            sys.exit("incorrect parameter: grid_type")


    X = np.meshgrid(*var_space)


    eff_ = []
    learn_max = []


    filename = optimization_method



    # initiate the search-accumulated array for GA
    if(optimizer=='geneticalgorithm'):
        # -------------------------------
        global GA_search_path,GA_search_top

        GA_search_path = []

        # to adjust the GA_parents
        # records the maximum objective of each search path
        # not the global maximum
        GA_search_top = [0]*GA_no_parents
        # -------------------------------

    if(optimization_method == 'hybrid_2' or optimization_method == 'hybrid_3'):

        global prob_model_r2
        prob_model_r2 = 0.



    opt_space = pd.DataFrame(columns=pd_header)

    for ivar in range(no_variables):
        opt_space[pd_header[ivar]] = X[ivar].flatten()


    mesh_points, tab_columns = opt_space.shape


    opt_space['probability'] = np.cumsum(np.ones(mesh_points))/np.cumsum(np.ones(mesh_points))[-1]
    opt_space['weighting'] = np.ones(mesh_points)



    if(readfile):
        opt_space = pd.read_csv(readfile)

        if(opt_space['probability'].isnull().values.any()):

            opt_space['weighting'] = np.ones(mesh_points)

            evaluated = opt_space[opt_space['objective'].notnull()].index.values
            opt_space['weighting'].loc[evaluated] = 0.


            prob_ = np.ones(mesh_points)
            prob_[evaluated] = 0.


            norm_prob_ = np.cumsum(prob_*opt_space['weighting'].as_matrix().flatten())
            opt_space['probability'] = norm_prob_ / norm_prob_[-1]



    opt_results = optimization(evaluate,optimizer,noIteration,opt_space,RF_depth,RF_estimator,no_selections,saveSpace)



    if(optimization_method == 'hybrid_2' or optimization_method == 'hybrid_3'):
        learn_max.append(learning_curve[-1])




    opt_results['objective'] = opt_results['objective'].fillna(value=-1E10)
    opt_res = opt_results[pd_header[:-2]].iloc[opt_results['objective'].idxmax()]  #.values



    return(opt_res,learning_curve)


In [5]:
def evaluate(x):
    
    #fitness = abs(-0.01*(-0.1*(x[0]-2.1)**3. + 0.8*(x[0]-2)**2. + 0.1*x[0] - 0.6) + 0.0005/(1. + (x[0]-2.005)) )
    fitness = abs(-np.sin(x[0]*6) + 0.05*x[0] + 0.02*(-(x[0]+0.51)**2.+4*x[0]+50.) )
    
    return np.around(fitness, decimals=10)

In [6]:
results = run("",[['linear',[0.0,10.,0.01]]],evaluate,'hybrid_2',int(10),int(200),"save")

In [7]:
# --------------
#
# select optimizer
# 'random','geneticalgorithm'(GA),'randomforest'(RF),
# 'hybrid_1' - 'GA-family' + 'RF-mc'
# 'hybrid_2' - 'random' + 'RF-wam'
# 'hybrid_3' - 'GA-family' + 'RF-wam'
# for GA: 'GA-family','GA-global'; 
# for RF: 'RF-wam'(whack-a-mole),'RF-mc'(monte-carlo)